# Load Libraries

In [ ]:
!pip install --upgrade httpx

In [2]:
from lxml import etree
import unicodedata
import re
from difflib import SequenceMatcher
from collections import defaultdict, Counter
import os
import anthropic

# Annotate with LLM

This Python script automatically transfers named entity annotations from a French TEI/XML document to its corresponding German translation using Anthropic’s Claude LLM (claude-sonnet-4-20250514). It aligns paragraph-by-paragraph and annotates the German text with TEI entity tags \<persName> and \<placeName> based on the French source.

In [3]:
import openai
from lxml import etree

# === STEP 1: Claude Setup ===
api_key = ""
client = anthropic.Anthropic(api_key=api_key)


french_path = "input/french_5-105.xml"
german_path = "input/german_5-105.xml"
output_path = "output/claude/german_annotated_5-105.xml"
debug_folder = "output/claude/debug"
os.makedirs(debug_folder, exist_ok=True)

ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

# === Load TEI Documents ===
french_tree = etree.parse(french_path)
german_tree = etree.parse(german_path)

french_paragraphs = french_tree.xpath('//tei:text//tei:p', namespaces=ns)
german_paragraphs = german_tree.xpath('//tei:text//tei:p', namespaces=ns)

# === XML sanity check ===
def is_valid_xml_snippet(text):
    text = text.strip()
    return text.startswith("<p") and text.endswith("</p>")

# === Claude Annotation Function ===
def annotate_german_with_claude(french_xml, german_text):
    prompt = (
        "You are an expert in TEI/XML and named entity annotation."
        "Given a paragraph in French (with named entities encoded using the TEI Tags <persName and <placeName>) "
        "and its German translation (not annotated), annotate exactly the same entities in the German text "
        "by wrapping them in the corresponding TEI tags. Encode persons and places."
        "ONLY encode these entities and structural things and nothing else; also map the correct"
        "ID from the @ref-Attribute. Sometimes the @ref can contain multiple IDs."
        "It is important to not encode additional entities." 
        "Do not change the german text itstelf."
        "Return only a single valid TEI XML element, starting with <p> and ending with </p>."
        "Also return the correct xml:id of the paragraph."
        "Do not include any commentary, code blocks, or explanations. The result must be valid XML.\n\n"
        f"French TEI/XML:\n{french_xml}\n\n"
        f"German Text:\n{german_text}\n\n"
        "Output (annotated German <p> element only):"
    )

    message = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=10000,
        temperature=0.0,
        system="You are a TEI/XML expert aligning named entity annotations from a French source to a German translation.",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    return message.content[0].text.strip()

# === Annotate Paragraphs ===
failures = []

for i, (fr_p, de_p) in enumerate(zip(french_paragraphs, german_paragraphs)):
    xml_id = de_p.attrib.get('{http://www.w3.org/XML/1998/namespace}id', f'p{i}')
    print(f"🔍 Aligning paragraph {xml_id}...")

    fr_xml = etree.tostring(fr_p, encoding="unicode")
    de_text = "".join(de_p.itertext()).strip()

    try:
        annotated_de = annotate_german_with_claude(fr_xml, de_text)

        if not is_valid_xml_snippet(annotated_de):
            print(f"Skipping invalid XML for {xml_id}: Not wrapped in <p>")
            failures.append((xml_id, "Output not wrapped in <p>"))
            with open(os.path.join(debug_folder, f"{xml_id}.txt"), "w", encoding="utf-8") as f:
                f.write(annotated_de)
            continue

        try:
            new_p = etree.fromstring(annotated_de.encode("utf-8"))
            parent = de_p.getparent()
            parent.replace(de_p, new_p)
        except Exception as parse_error:
            print(f"XML parsing failed for {xml_id}: {parse_error}")
            failures.append((xml_id, f"Invalid XML from Claude: {parse_error}"))
            with open(os.path.join(debug_folder, f"{xml_id}.txt"), "w", encoding="utf-8") as f:
                f.write(annotated_de)
            continue

    except Exception as e:
        print(f"❌ Claude call failed for {xml_id}: {e}")
        failures.append((xml_id, str(e)))
        continue

# === Save Annotated German TEI ===
german_tree.write(output_path, encoding="utf-8", xml_declaration=True, pretty_print=True)
print(f"\nAnnotated German TEI saved to: {output_path}")

# === Report Failures ===
if failures:
    print("\nThese paragraphs failed:")
    for fid, reason in failures:
        print(f"- {fid}: {reason}")

print(f"\nFinished: {len(german_paragraphs) - len(failures)} succeeded, {len(failures)} failed out of {len(german_paragraphs)} paragraphs.")


🔍 Aligning paragraph P.5...
🔍 Aligning paragraph P.6...
🔍 Aligning paragraph P.7...
🔍 Aligning paragraph P.8...
🔍 Aligning paragraph P.9...
🔍 Aligning paragraph P.10...
🔍 Aligning paragraph P.11...
🔍 Aligning paragraph P.12...
🔍 Aligning paragraph P.13...
🔍 Aligning paragraph P.14...
🔍 Aligning paragraph P.15...
🔍 Aligning paragraph P.16...
🔍 Aligning paragraph P.17...
🔍 Aligning paragraph P.18...
🔍 Aligning paragraph P.19...
🔍 Aligning paragraph P.20...
🔍 Aligning paragraph P.21...
🔍 Aligning paragraph P.22...
🔍 Aligning paragraph P.23...
🔍 Aligning paragraph P.24...
🔍 Aligning paragraph P.25...
🔍 Aligning paragraph P.26...
🔍 Aligning paragraph P.27...
🔍 Aligning paragraph P.28...
🔍 Aligning paragraph P.29...
🔍 Aligning paragraph P.30...
🔍 Aligning paragraph P.31...
🔍 Aligning paragraph P.32...
🔍 Aligning paragraph P.33...
🔍 Aligning paragraph P.34...
🔍 Aligning paragraph P.35...
🔍 Aligning paragraph P.36...
🔍 Aligning paragraph P.37...
🔍 Aligning paragraph P.38...
🔍 Aligning paragrap

# Inspect the annotated german TEI/XML-File

This script analyzes the automatic TEI-encoded XML document to identify all unique XML element types that appear within \<p> (paragraph) elements. It is useful for understanding the variety of tags used in annotated text, particularly when verifying or analyzing entity annotations (because sometimes LLMs tend to use different tags - for example \<lem> and \<rdg> instead of two \<rdg> inside an \<app>.

In [7]:
# Load the XML content
with open("output/claude/german_annotated_5-105.xml", "rb") as f:
    tree = etree.parse(f)

# TEI namespace (adjust if needed)
namespaces = {'tei': 'http://www.tei-c.org/ns/1.0'}

# Find all <p> elements
p_elements = tree.xpath("//tei:p", namespaces=namespaces)

# Collect all unique element tags used inside <p> elements
unique_tags = set()

for p in p_elements:
    for el in p.iterdescendants():
        # Remove namespace prefix if present
        tag = el.tag.split('}')[-1]
        unique_tags.add(tag)

# Print results
print("Unique elements inside <p> elements:")
for tag in sorted(unique_tags):
    print(f"- <{tag}>")


Unique elements inside <p> elements:
- <anchor>
- <bibl>
- <hi>
- <persName>
- <placeName>


# Evaluation

This Python script analyzes and compares named entity annotations (\<persName>, \<placeName>) in two aligned TEI-encoded XML files: a French source file and its German translation. It extracts all entity references (@ref) per paragraph, identifies missing or extra references in the German version relative to the French source, and reports discrepancies. The script also logs and summarizes empty @ref attributes and categorizes differences by entity type. 

In [ ]:
from lxml import etree
from collections import Counter
from html import escape

# === Dateien und Namespace ===
french_file = "input/french_5-105.xml"
german_file = "output/claude/all_annotations/german_annotated_5-105.xml"
ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

# === Hilfsfunktion: Typ eines @ref bestimmen ===
def get_ref_type(ref, tree):
    for e in tree.xpath(f'//tei:persName[@ref="{ref}"]', namespaces=ns):
        return "persName"
    for e in tree.xpath(f'//tei:placeName[@ref="{ref}"]', namespaces=ns):
        return "placeName"
    return "unknown"

# === Daten extrahieren mit ref-Zählung ===
def extract_entity_refs_and_text(tree):
    entity_map = {}
    text_map = {}
    empty_ref_log = []
    valid_entity_counter = Counter()

    for p in tree.xpath('//tei:p[@xml:id]', namespaces=ns):
        xml_id = p.attrib.get('{http://www.w3.org/XML/1998/namespace}id')
        ref_counter = Counter()

        for e in p.xpath('.//tei:persName|.//tei:placeName', namespaces=ns):
            tag = e.tag.split('}')[-1]
            ref = e.attrib.get('ref')
            entity_text = "".join(e.itertext()).strip()

            if not ref or not ref.strip():
                empty_ref_log.append({
                    'paragraph': xml_id,
                    'element': tag,
                    'text': entity_text
                })
            else:
                ref_counter[ref.strip()] += 1
                valid_entity_counter[tag] += 1

        entity_map[xml_id] = ref_counter
        text_map[xml_id] = "".join(p.itertext()).strip()

    return entity_map, text_map, empty_ref_log, valid_entity_counter

# === Parsen ===
french_tree = etree.parse(french_file)
german_tree = etree.parse(german_file)

french_entities, french_texts, french_empty_refs, french_entity_count = extract_entity_refs_and_text(french_tree)
german_entities, german_texts, german_empty_refs, _ = extract_entity_refs_and_text(german_tree)

# === Abgleich Refs mit Zählung ===
discrepancies = []
count_missing = 0
count_extra = 0
missing_counter = Counter()
extra_counter = Counter()
common_ids = set(french_entities) & set(german_entities)

for para_id in sorted(common_ids):
    fr_counter = french_entities[para_id]
    de_counter = german_entities[para_id]

    all_refs = set(fr_counter.keys()) | set(de_counter.keys())
    extra = []
    missing = []

    for ref in sorted(all_refs):
        fr_count = fr_counter.get(ref, 0)
        de_count = de_counter.get(ref, 0)

        if fr_count > de_count:
            diff = fr_count - de_count
            missing.append(f"{ref} (×{diff})")
            ref_type = get_ref_type(ref, french_tree)
            missing_counter[ref_type] += diff
        elif de_count > fr_count:
            diff = de_count - fr_count
            extra.append(f"{ref} (×{diff})")
            ref_type = get_ref_type(ref, german_tree)
            extra_counter[ref_type] += diff

    if extra or missing:
        discrepancies.append({
            'paragraph': para_id,
            'extra_in_german': extra,
            'missing_in_german': missing,
            'french_text': french_texts.get(para_id, ""),
            'german_text': german_texts.get(para_id, "")
        })
        if extra:
            count_extra += 1
        if missing:
            count_missing += 1

# === Neue Prüfung: Inkonsistente Tag-Typen ===
tag_mismatches = []

for para_id in sorted(common_ids):
    fr_elements = french_tree.xpath(f'//tei:p[@xml:id="{para_id}"]//tei:persName | //tei:p[@xml:id="{para_id}"]//tei:placeName', namespaces=ns)
    de_elements = german_tree.xpath(f'//tei:p[@xml:id="{para_id}"]//tei:persName | //tei:p[@xml:id="{para_id}"]//tei:placeName', namespaces=ns)

    fr_tags_by_ref = {e.attrib.get('ref'): e.tag.split('}')[-1] for e in fr_elements if e.attrib.get('ref')}
    de_tags_by_ref = {e.attrib.get('ref'): e.tag.split('}')[-1] for e in de_elements if e.attrib.get('ref')}

    common_refs = set(fr_tags_by_ref) & set(de_tags_by_ref)

    for ref in common_refs:
        fr_tag = fr_tags_by_ref[ref]
        de_tag = de_tags_by_ref[ref]
        if fr_tag != de_tag:
            tag_mismatches.append({
                'paragraph': para_id,
                'ref': ref,
                'french_tag': fr_tag,
                'german_tag': de_tag,
                'french_text': french_texts.get(para_id, ""),
                'german_text': german_texts.get(para_id, "")
            })

# === Zusammenfassung ===
total_paragraphs = len(common_ids)
total_discrepant = len(discrepancies)
total_french_empty = len(french_empty_refs)
total_german_empty = len(german_empty_refs)

def summarize_empty_refs(ref_list):
    counter = Counter()
    for item in ref_list:
        counter[item['element']] += 1
    return counter

# === Zusammenfassungsbericht ===
print("=== 🔍 Annotation Summary Report ===")
print(f"Total aligned paragraphs:         {total_paragraphs}")
print(f"Paragraphs with missing refs:     {count_missing}")
print(f"Paragraphs with extra refs:       {count_extra}")
print(f"Paragraphs with any discrepancies:{total_discrepant}")
print()
print(f"Valid French entities with @ref:")
print(f"  👤 persName:  {french_entity_count['persName']}")
print(f"  📍 placeName: {french_entity_count['placeName']}")
print()
print(f"Empty @ref attributes (FR):       {total_french_empty}")
print(f"Empty @ref attributes (DE):       {total_german_empty}")
print("Empty @ref breakdown (FR):", summarize_empty_refs(french_empty_refs))
print("Empty @ref breakdown (DE):", summarize_empty_refs(german_empty_refs))
print()

# === Neue Auswertung: Totale Differenzen nach Typ ===
print("=== Totals for Missing/Extra Entities ===")
print(f"❌ Missing in German:")
print(f"   👤 persName:  {missing_counter['persName']}")
print(f"   📍 placeName: {missing_counter['placeName']}")
print(f"➕ Extra in German:")
print(f"   👤 persName:  {extra_counter['persName']}")
print(f"   📍 placeName: {extra_counter['placeName']}")
print("=========================================")

# === Details zu Differenzen ===
if discrepancies:
    print("=== Entity Annotation Discrepancies ===")
    for d in discrepancies:
        print(f"\n📌 Paragraph {d['paragraph']}:")
        if d['extra_in_german']:
            print(f"  ➕ Extra in German:  {d['extra_in_german']}")
        if d['missing_in_german']:
            print(f"  ❌ Missing in German: {d['missing_in_german']}")
        print("  🇫🇷 French text:\n ", d['french_text'][:400], "...\n")
        print("  🇩🇪 German text:\n ", d['german_text'][:400], "...\n")
else:
    print("✅ No discrepancies in entity annotations.")

# === Bericht zu Tag-Inkonsistenzen ===
if tag_mismatches:
    print("=== ⚠️ Tag Type Mismatches (persName vs placeName) ===")
    for m in tag_mismatches:
        print(f"\n📌 Paragraph {m['paragraph']}:")
        print(f"  🔄 Ref: {m['ref']}")
        print(f"  🇫🇷 French tag: <{m['french_tag']}>")
        print(f"  🇩🇪 German tag: <{m['german_tag']}>")
        print("  🇫🇷 French text:\n ", m['french_text'][:400], "...\n")
        print("  🇩🇪 German text:\n ", m['german_text'][:400], "...\n")
else:
    print("✅ No tag type mismatches found.")

# === Leere Refs ===
if french_empty_refs or german_empty_refs:
    print("\n=== Elements with Empty @ref Attributes ===")
    for item in french_empty_refs:
        print(f"[FR] Paragraph {item['paragraph']} — <{item['element']}>: '{item['text']}' has no @ref")
    for item in german_empty_refs:
        print(f"[DE] Paragraph {item['paragraph']} — <{item['element']}>: '{item['text']}' has no @ref")
else:
    print("✅ No empty @ref attributes found.")
